<h2 align= "center">Part 2</h2>

#### Question 1: Check if the sentence is Grammatically correct: Please use any pre-trained model or use text from open datasets. Once done, please evaluate the English Grammar in the text column of the below dataset.

Dataset Link: https://drive.google.com/file/d/1LTI5KNqPrtxrYRgJk2AxI30KgYyNcRpD/view

In [11]:
!pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Using  a fine-tuned version of Google's T5 model. T5 is a text-to-text model, meaning given text, it produces a standalone piece of text. It is currently considered "state-of-the-art," and the largest model even outperforms the human baseline on the General Language Understanding Evaluation benchmark.

In [12]:
from happytransformer import  HappyTextToText
from difflib import SequenceMatcher
from happytransformer import TTSettings
import pandas as pd
import numpy as np

In [13]:
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

06/22/2022 12:41:10 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [14]:
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=100)

In [15]:
def Grammer_score(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [16]:
df = pd.read_csv('/content/review_data.csv')
df = df.sample(frac=0.003) #sampling a fraction of data to reduce time
df.head()

,text,star,app_id,reviewDate
1040,I can download some of the music which I can't...,1,com.tencent.ibg.joox,18/03/21
14611,Superb app,5,com.et.market,19/03/21
27805,Good,5,jp.konami.pesam,21/03/21
29157,Related with junta myanmar.,1,com.tencent.ibg.joox,21/03/21
3867,Amazing,5,com.camerasideas.instashot,18/03/21


In [17]:
correct = []
for text in df.text:
  output = happy_tt.generate_text("grammar: " + text,args=beam_settings)
  correct.append(output.text)
df['Correct_text'] = correct

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [18]:
score = []
for text,Correct_text in zip(df.text,df.Correct_text):
  score.append(Grammer_score(text,Correct_text))
df['score'] = score

In [19]:
df['Grammer'] = np.where(df['score'] > 0.9 , "Correct", "wrong")

In [20]:
df.head()

,text,star,app_id,reviewDate,Correct_text,score,Grammer
1040,I can download some of the music which I can't...,1,com.tencent.ibg.joox,18/03/21,I can download some of the music which I can't...,0.986667,Correct
14611,Superb app,5,com.et.market,19/03/21,Superb app.,0.952381,Correct
27805,Good,5,jp.konami.pesam,21/03/21,Good luck! Good luck! Good luck! Good luck! Go...,0.027682,wrong
29157,Related with junta myanmar.,1,com.tencent.ibg.joox,21/03/21,Related with junta myanmar.,1.000000,Correct
3867,Amazing,5,com.camerasideas.instashot,18/03/21,Amazing! Amazing! Amazing! Amazing! Amazing! A...,0.034826,wrong
